## Building a dataset, training a Seq2Seq model, and testing it

The Virginia Tech Natural Motion Dataset contains .h5 files with unscripted human motion data collected in real-world environments as participants went about their day-to-day lives. This is a brief tutorial in using the dataset and then training and testing a neural network.

This tutorial illustrates how to use the shell (.sh) scripts to train a seq2seq model (particularly **train_seq2seq.sh** and **test_seq2seq.sh**). Similar shell scripts are also available for the Transformers (see **train_transformer.sh** and **test_transformer.sh**)

### Building a dataset

We will first cover how to build a dataset with data from a few participants using the build-dataset.py file.

We are running the script from a Jupyter Notebook, but this can just as easily be run as a shell script (see build_dataset.sh).

In this case, we are drawing data from the h5-dataset folder located in the cloud. We are going to output the training.h5, validation.h5, and testing.h5 files to the folder data/set-2.

We will be using participants 1, 5, and 10 (P1, P5, P10, respectively) and extracting normOrientation and normAcceleration data on a few segments (norm* means data normalized relative to the pelvis). As output data we will be extracting normOrientation data for every segment.

In other words, our task is as follows: use orientation and acceleration from a set of sparse segments and try to train a model mapping that input data to orientations for every segment on the human body.

In [1]:
!mkdir -p /home/jackg7/VT-Natural-Motion-Processing/data/set-2
!python build-dataset.py --data-path "/groups/MotionPred/h5-dataset" \
                         --output-path "/home/jackg7/VT-Natural-Motion-Processing/data/set-2" \
                         --training "P1" \
                         --validation "P5" \
                         --testing "P10" \
                         --task-input "normOrientation normAcceleration" \
                         --input-label-request "T8 RightForeArm RightLowerLeg LeftForeArm LeftLowerLeg" \
                         --task-output "normOrientation" \
                         --output-label-request "all"

2020-08-10 13:09:31 INFO     Writing X to the training file group...
2020-08-10 13:09:37 INFO     Writing X to the validation file group...
2020-08-10 13:09:40 INFO     Writing X to the testing file group...
2020-08-10 13:09:50 INFO     Writing Y to the training file group...
2020-08-10 13:09:57 INFO     Writing Y to the validation file group...
2020-08-10 13:09:58 INFO     Writing Y to the testing file group...


### Training a seq2seq model

We can now train a seq2seq model to map the normOrientation and normAcceleration data from the sparse segments to the full-body normOrientation data.

We will be using a seq-length of 30 (at 240 Hz) downsample it by a factor of 6 (to 40 Hz). The resulting sequences will be of length 5 for the input and output. The in-out-ratio will then be used to reduce the output sequence length to 1.

The input sequence will be of shape (B, 5, 35) and output shape will be of shape (B, 1, 92). Orientations are stored as quaternions, so orientation value will be 4 in length. The number 35 comes from our use of 5 segment orientations and accelerations or $5*4 + 5*3 = 35$. The full-body has 23 segments and we're predicting orientation values for each one or $23*4 = 92$

We're training a seq2seq model with a hidden size of 512, a bidirectional encoder and dot product attention. The model will be trained for a single epoch.

Our loss function for training will be the L1Loss and our validation losses will be the L1Loss and the QuatDistance (cosine similarity) loss.

In [2]:
!mkdir -p /home/jackg7/VT-Natural-Motion-Processing/models/set-2
!python train-seq2seq.py --task conversion \
                         --data-path "/home/jackg7/VT-Natural-Motion-Processing/data/set-2" \
                         --model-file-path "/home/jackg7/VT-Natural-Motion-Processing/models/set-2/model.pt" \
                         --representation quaternions \
                         --batch-size=32 \
                         --seq-length=30 \
                         --downsample=6 \
                         --in-out-ratio=5 \
                         --stride=30 \
                         --learning-rate=0.001 \
                         --num-epochs=1 \
                         --hidden-size=512 \
                         --attention=dot \
                         --bidirectional

2020-08-10 13:10:06 INFO     task - conversion
2020-08-10 13:10:06 INFO     data_path - /home/jackg7/VT-Natural-Motion-Processing/data/set-2
2020-08-10 13:10:06 INFO     model_file_path - /home/jackg7/VT-Natural-Motion-Processing/models/set-2/model.pt
2020-08-10 13:10:06 INFO     representation - quaternions
2020-08-10 13:10:06 INFO     auxiliary_acc - False
2020-08-10 13:10:06 INFO     batch_size - 32
2020-08-10 13:10:06 INFO     learning_rate - 0.001
2020-08-10 13:10:06 INFO     seq_length - 30
2020-08-10 13:10:06 INFO     downsample - 6
2020-08-10 13:10:06 INFO     in_out_ratio - 5
2020-08-10 13:10:06 INFO     stride - 30
2020-08-10 13:10:06 INFO     num_epochs - 1
2020-08-10 13:10:06 INFO     hidden_size - 512
2020-08-10 13:10:06 INFO     dropout - 0.0
2020-08-10 13:10:06 INFO     bidirectional - True
2020-08-10 13:10:06 INFO     attention - dot
2020-08-10 13:10:06 INFO     Starting seq2seq model training...
2020-08-10 13:10:06 INFO     Retrieving training data for sequences 125 ms

### Testing our model

We can now test our model and output a histogram of performance over the testing data. The model parameters must be the same to properly read in the model.

In [3]:
!mkdir -p /home/jackg7/VT-Natural-Motion-Processing/images
!python test-seq2seq.py  --task conversion \
                         --data-path-parent /home/jackg7/VT-Natural-Motion-Processing/data \
                         --figure-file-path /home/jackg7/VT-Natural-Motion-Processing/images/seq2seq-test.pdf \
                         --figure-title "Seq2Seq" \
                         --model-dir /home/jackg7/VT-Natural-Motion-Processing/models/set-2 \
                         --representation quaternions \
                         --batch-size=512 \
                         --seq-length=30 \
                         --downsample=6 \
                         --in-out-ratio=5 \
                         --stride=30 \
                         --hidden-size=512 \
                         --attention=dot \
                         --bidirectional

2020-08-10 13:14:06 INFO     task - conversion
2020-08-10 13:14:06 INFO     data_path_parent - /home/jackg7/VT-Natural-Motion-Processing/data
2020-08-10 13:14:06 INFO     figure_file_path - /home/jackg7/VT-Natural-Motion-Processing/images/seq2seq-test.pdf
2020-08-10 13:14:06 INFO     figure_title - Seq2Seq
2020-08-10 13:14:06 INFO     include_legend - False
2020-08-10 13:14:06 INFO     model_dir - /home/jackg7/VT-Natural-Motion-Processing/models/set-2
2020-08-10 13:14:06 INFO     representation - quaternions
2020-08-10 13:14:06 INFO     batch_size - 512
2020-08-10 13:14:06 INFO     seq_length - 30
2020-08-10 13:14:06 INFO     downsample - 6
2020-08-10 13:14:06 INFO     in_out_ratio - 5
2020-08-10 13:14:06 INFO     stride - 30
2020-08-10 13:14:06 INFO     hidden_size - 512
2020-08-10 13:14:06 INFO     dropout - 0.0
2020-08-10 13:14:06 INFO     bidirectional - True
2020-08-10 13:14:06 INFO     attention - dot
2020-08-10 13:14:06 INFO     Starting seq2seq model testing...
2020-08-10 13:14

We can now visualize the performance of the seq2seq model on the test data.

In [4]:
from IPython.display import IFrame
IFrame("../images/seq2seq-test.pdf", width=600, height=300)